In [ ]:
pip install autogen-agentchat autogen-ext[openai] google-search-results serpapi

In [ ]:
import asyncio
from serpapi.google_search import GoogleSearch
from google.colab import userdata

SERPER_API_KEY = userdata.get('SERPER_API_KEY')

async def get_hotels(departure_city: str,start_date: str,end_date: str) -> str:
  '''
  Get hotels for a given departure_city, start date and end date.
  The response is a list of hotels
  '''

  ## print(f'---------------- input received: {departure_city}, {start_date} and {end_date}')

  params = {
  "engine": "google_hotels",
  "q": departure_city,
  "check_in_date": start_date,
  "check_out_date": end_date,
  "adults": "2",
  "currency": "INR",
  "gl": "in",
  "hl": "en",
  "api_key": SERPER_API_KEY
  }

  search = await asyncio.to_thread(lambda: GoogleSearch(params))
  results = search.get_dict().get('properties')
  ## print(f'------------------- hotel results: {results}')
  return results

In [ ]:
import asyncio
from serpapi.google_search import GoogleSearch
from google.colab import userdata

SERPER_API_KEY = userdata.get('SERPER_API_KEY')

async def get_flights(departure_city: str, arrival_city: str, start_date: str,end_date: str) -> str:
  '''
  Get flights for a given departure city, arrival city, start date and end date.
  The response is a list of flights
  '''

  ## print(f'---------------- input received: {departure_city}, {arrival_city}, {start_date} and {end_date}')

  city_codes = {'bangalore': 'BLR', 'delhi': 'DEL', 'kolkata': 'CCU', 'hyderabad': 'HYD'}

  departure_code = city_codes.get(departure_city.lower())
  arrival_code = city_codes.get(arrival_city.lower())

  params = {
  "engine": "google_flights",
  "departure_id": departure_code,
  "arrival_id": arrival_code,
  "outbound_date": start_date,
  "return_date": end_date,
  "currency": "INR",
  "hl": "en",
  "api_key": SERPER_API_KEY
  }

  search = await asyncio.to_thread(lambda: GoogleSearch(params))
  results = search.get_dict().get('other_flights')
  ##print(f'------------------- flight results: {results}')
  return results


In [ ]:
recommender_prompt= '''
        **AI Travel Planner**
        You are an experienced travel agent who can create a detailed itinerary for customers. Based on the flight and hotel information, create a {days}-day itinerary for the user:

        The itinerary should include:
            - Summarize flight details in a table format and show only the top 3 flights based on best price, shorter travel time and convinient travel time.
            - Summarize hotel details in a table format and show only the top 3 hotels based on best price, ratings and good amenities.
            - Day-by-day breakdown of activities that are recommended for each day of the trip.
            - Must-visit attractions and estimated visit times
            - Restaurant recommendations for meals
            - Tips for local transportation

            **Format Requirements**:
            - Use markdown formatting with clear headings (# for main headings, ## for days, ### for sections)
            - Include emojis for different types of activities ( for landmarks, 🍽️ for restaurants, etc.)
            - Use bullet points for listing activities
            - Include estimated timings for each activity
            - Format the itinerary to be visually appealing and easy to read.

        Once you are done with the recommendation, please add "TERMINATE" in the end of the message.
    '''

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from google.colab import userdata
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

model_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=GOOGLE_API_KEY,
)

agent1 = AssistantAgent(
    name="AI_Hotel_Analyst",
    model_client=model_client,
    tools=[get_hotels],
    system_message="""
        As a hotel analyst, find which is the arrival city for the trip from user input. Make relevant tool call and get hotel details for the arrival city, start date and end date.
        """
)

agent2 = AssistantAgent(
    name="AI_Flight_Analyst",
    model_client=model_client,
    tools=[get_flights],
    system_message="""
        Make the relevant tool call and get flight details for given departure city, arrival city, start date and end date.
        """
)

agent3 = AssistantAgent(
    name = "AI_Travel_Planner",
    model_client=model_client,
    system_message= recommender_prompt
)

team = RoundRobinGroupChat([agent1, agent2, agent3], termination_condition=TextMentionTermination("TERMINATE"))

await Console(team.run_stream(task="Help me plan a trip from Bangalore to Delhi for 2025-11-14 to 2025-11-17"))

---------- TextMessage (user) ----------
Help me plan a trip from Bangalore to Delhi for 2025-11-14 to 2025-11-17
---------- ToolCallRequestEvent (AI_Hotel_Analyst) ----------
[FunctionCall(id='function-call-4961630875187450495', arguments='{"end_date":"2025-11-17","start_date":"2025-11-14","departure_city":"Delhi"}', name='get_hotels')]
---------- ToolCallExecutionEvent (AI_Hotel_Analyst) ----------
[FunctionExecutionResult(content='[{\'type\': \'hotel\', \'name\': \'The Lalit New Delhi\', \'description\': \'Sophisticated rooms & suites in a posh lodging featuring 6 eateries, a nightclub & a spa.\', \'link\': \'https://www.thelalit.com/the-lalit-delhi/\', \'property_token\': \'ChgIuJmksoPYirU5GgwvZy8xMnE0d2Y1OGQQAQ\', \'serpapi_property_details_link\': \'https://serpapi.com/search.json?adults=2&check_in_date=2025-11-14&check_out_date=2025-11-17&children=0&currency=INR&engine=google_hotels&gl=in&hl=en&property_token=ChgIuJmksoPYirU5GgwvZy8xMnE0d2Y1OGQQAQ&q=Delhi\', \'gps_coordinates\':

TaskResult(messages=[TextMessage(id='e99ecf6a-1ac4-48e0-a8fa-30d1aca73bb1', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 25, 7, 3, 4, 920508, tzinfo=datetime.timezone.utc), content='Help me plan a trip from Bangalore to Delhi for 2025-11-14 to 2025-11-17', type='TextMessage'), ToolCallRequestEvent(id='7a653453-1f23-42de-98ea-7b9a7a6ef083', source='AI_Hotel_Analyst', models_usage=RequestUsage(prompt_tokens=174, completion_tokens=49), metadata={}, created_at=datetime.datetime(2025, 10, 25, 7, 3, 7, 164105, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='function-call-4961630875187450495', arguments='{"end_date":"2025-11-17","start_date":"2025-11-14","departure_city":"Delhi"}', name='get_hotels')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='17621ece-d539-49ae-8ee1-cb4932e8ad43', source='AI_Hotel_Analyst', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 25, 7, 3, 7, 224786, tzinfo=datetime.timezone.utc),